In [ ]:
# External libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
import matplotlib.style as style
import cv2
import time
import os
from utility.utils import *
import re
import h5py
import tables

# Own Libraries
from utility.utils import *
from Match.pair import *
from Match.icp import *

%matplotlib tk
style.use('seaborn-paper')

style.use('seaborn-paper')
%matplotlib tk

In [ ]:
filepath= 'DATA/lunar_crater_database_robbins_2018.csv'
df = pd.read_csv(filepath, sep=',')
df = df[ ['LON_CIRC_IMG','LAT_CIRC_IMG','DIAM_CIRC_IMG']  ] # Select columns
df['LON_CIRC_IMG'] = np.where(df['LON_CIRC_IMG'] > 180, df['LON_CIRC_IMG'] - 360, df['LON_CIRC_IMG']) 
df['DIAM_CIRC_IMG'] = df['DIAM_CIRC_IMG'] 

In [ ]:
df.rename(columns={'LON_CIRC_IMG':'lon','LAT_CIRC_IMG':'lat','DIAM_CIRC_IMG':'r'}, inplace=True)

sort_lat = df.sort_values(by='lat')
sort_lat.insert(3, 'i', range(len(sort_lat)))

sort_lat_lon = sort_lat.sort_values(by='lon')
sort_lat_lon.insert(4,'j', range(len(sort_lat_lon)))

df_sort = sort_lat_lon.sort_index()

In [ ]:
# height_filter_catalog
H = 50
FOV_half = np.deg2rad(90/2)
d = 2*H* np.tan(FOV_half)
filter = 2/3*d

df_sort_f = df_sort[df_sort['r'] < filter].copy()
df_sort_f = df_sort_f.reset_index(drop=True)

In [ ]:
crt1, crt2, crt3, PICKS = find_triplet( df_sort_f, 0)
max_crt_dist = np.max([dist_crt(crt1,crt2),dist_crt(crt2,crt3),dist_crt(crt1,crt3)])
# print(max_crt_dist) # Into the query search: filter all triplet with max_crt_dist > d

triplet = [crt1, crt2, crt3]
K_vet = compute_K_vet(triplet)
print(K_vet)
Lat_bounds = [np.min([crt1.lat,crt2.lat,crt3.lat]),  np.max([crt1.lat,crt2.lat,crt3.lat]) ]
Lon_bounds = [np.min([crt1.lon,crt2.lon,crt3.lon]),  np.max([crt1.lon,crt2.lon,crt3.lon]) ]
entry = [ Lat_bounds, Lon_bounds, K_vet, [crt1, crt2, crt3]]
print(entry)


In [ ]:
# Crater_TRN_catalog = pd.DataFrame({
# 'Lat_bounds': Lat_bounds,
# 'Lon_bounds': Lon_bounds,
# 'Max ij-crt distance': max_crt_dist,
# })

In [ ]:
entry_pd = pd.Series(entry)

In [ ]:
Crater_TRN_catalog = pd.DataFrame({'triplet':entry_pd})

In [ ]:
Crater_TRN_catalog

# CATALOG CREATION

In [ ]:
df = pd.read_csv('DATA/lunar_crater_database_robbins_2018.csv')
df = df[['LAT_CIRC_IMG','LON_CIRC_IMG','DIAM_CIRC_IMG']]
df.rename( columns= {'LAT_CIRC_IMG':'lat', 'LON_CIRC_IMG':'lon','DIAM_CIRC_IMG':'r'}, inplace=True )
df.r = df.r/2
df.lon = np.where(df.lon < 180, df.lon, df.lon - 360)
df = df.sort_values(by='lat')
df.insert(3,'i',range(len(df)))
df = df.sort_values(by='lon')
df.insert(4,'j',range(len(df)))
df = df[df.r < 100]
df.reset_index(drop=True, inplace=True)

In [ ]:
# init H
H = pd.DataFrame({
'Lat_min':[0],
'Lat_max':[0],
'Lon_min':[0],
'Lon_max':[0],
'ICD_max':[0],
'angle1':[0],
'angle2':[0],
'angle3':[0],
'crt_name1':[0],
'crt_name2':[0],
'crt_name3':[0]
})

In [ ]:
for ind in range(400000,500000):
    triplet = find_triplet(df, ind)
    K_v = compute_K_vet(triplet)
    ICD_max = find_max_crt_dist(triplet)
    lat_m, lat_M, lon_m, lon_M = find_bounds(triplet)
    R_M = find_max_radius(triplet)
    entry = [lat_m, lat_M, lon_m, lon_M, ICD_max, K_v[0], K_v[1], K_v[2], triplet[0].name,triplet[1].name,triplet[2].name]
    if np.all(K_v > 15):
        H.loc[ind] = entry
    printProgressBar(ind, df.shape[0], printEnd='')    
    file_name = 'TRN_catalog_part_B.csv'
    H.to_csv(file_name)
    print(ind)
    

In [ ]:
# def image_gen(PATH):
PATH = 'DATA/ephemeris sat/20 step size/IMG_20_Lat_0_Long_-127.683.jpg'
path = '/home/sirbastiano/Desktop/Python Projects/Progetto Tesi/'+PATH

# Actual Position:
num =20
l = -127.683
H = 50
X_c, Y_c = l, 0
# x_f, y_f, z_f = spherical2cartesian(0, crater.Lat, crater.Lon)

# Find reference in catalogs:
dteta = find_dteta(H) # d/R, where d^2 is the footprint
sp = dteta/2 # span
lat_bounds, lon_bounds = np.array([Y_c-sp,Y_c+sp]), np.array([X_c-sp,X_c+sp])
print('*'*35 +'BORDERS'+'*'*35)
print(f'LAT: {lat_bounds}              LON:{lon_bounds}')
print('*'*35 +'*******'+'*'*35)
# 3 crater catalogs: HEAD | LROC | ROBBINS
CATALOGS = ['ROBBINS', 'HEAD', 'LROC']
for DATA in CATALOGS:
    if DATA =='HEAD':
        filepath = '/home/sirbastiano/Desktop/Python Projects/Progetto Tesi/DATA/HeadCraters.csv'
    elif DATA == 'LROC':
        filepath = '/home/sirbastiano/Desktop/Python Projects/Progetto Tesi/DATA/LROCCraters.csv'
    elif DATA == 'ROBBINS':
        filepath= '/home/sirbastiano/Desktop/Python Projects/Progetto Tesi/DATA/lunar_crater_database_robbins_2018.csv'

    DB = pd.read_csv(filepath, sep=',')
    df = CatalogSearch(DB, lat_bounds, lon_bounds, CAT_NAME=DATA)
    if df is None:
        print('No Craters found!')
    else:     
        print(f'Found {df.shape[0]} craters!\n\n', df)
    
    ## DRAWINGS: 
    ## FIG.1
    img = cv2.imread(path)
    plt.ion()
    plt.figure('Tile and Mask -'+DATA, dpi=200)
    plt.subplot(1,2,1)
    plt.imshow(img)
    plt.subplot(1,2,2)
    I = draw_craters(df, lon_bounds, lat_bounds)
    plt.imshow(I)
    plt.show()
    plt.savefig(DATA+str(num)+'_fig-A.png')
    
    ## FIG.2
    image_with_craters = draw_craters_on_image(df,  lon_bounds, lat_bounds, img, u=None)
    plt.ion()
    plt.figure('Image with craters catalogued-'+DATA, dpi=200)
    plt.imshow(image_with_craters)
    plt.show()
    plt.savefig(DATA+str(num)+'_fig-B.png')

plt.close('all')

In [ ]:
PATHS = os.listdir("/home/sirbastiano/Desktop/Python Projects/Progetto Tesi/DATA/ephemeris sat/20 step size")
print(PATHS)